# Pseudorandomness, Sets and Permutations

**Introduction to Pseudorandomness**

Nothing a computer can do is ever truly random. Through clever use of mathematics (calculations) and physics (readings from sensors such as temperature), it is possible to give the impression of randomness. With more work, even statistically convincing randomness is viable to some degree.

Most programming languages contain some routines for (pseudo-)random number generation within their built-in or default packages, whereas the finer ones (with hope of statistical validity, although not necessarily cryptographic strength) tend to be included within additional optional libraries that are installed as needed.

1. **Basic Random Number Generation**

In [1]:
from random import random

print(random())  # Output: A random float between 0.0 and 1.0

0.2660545280944506


2. **Boolean Values with Randomness**

**Generating Random Boolean Values**

In [2]:
n = 10
print([random() < 0.5 for k in range(n)])  # Output: A list of True/False values

[False, True, False, False, True, False, False, False, False, True]


**Count of Trues in a Random Boolean List**

In [3]:
n = 10000
print(sum([random() < 0.5 for k in range(n)]))  # Output: An approximate count around 5000

5079


3. **Random Values within an Interval**

**Generating Random Values in a Range**


In [4]:
from random import uniform

r = 10
a = -10
b = 25

print([uniform(a, b) for _ in range(r)])  # Output: Random floats within [-10, 25]

[2.3549028068211886, 15.745563831370472, 24.33500663919898, 11.25703502610719, 15.289635061080233, 5.8335892520803245, 17.937648533399692, -5.140086960396356, -7.938237179957008, 20.616741214062504]


**Custom Function for Random Values**

In [5]:
def myown(low, high):
    span = high - low
    return low + span * random()

print([myown(a, b) for _ in range(r)])  # Output: Similar to uniform(a, b)

[5.278486061621814, 5.118564032393511, -5.447223756123051, 20.499524847180385, 21.67770669502503, 19.5756416955414, 20.580057567643273, 23.44238233913068, 16.114604397586966, 10.742033687223959]


4. **Random Integers**

**Generating Random Integers**


In [6]:
from random import randrange

r = 10
a = -10
b = 25

print([randrange(a, b + 1) for _ in range(r)])  # Output: Random integers within [-10, 25]

[22, 22, 25, 14, -7, 6, -1, -10, 23, 0]


5. **Selecting Random Elements**

**Picking Random Elements from a List**

In [7]:
data = [5, 19, 67, 24]
from random import choice

def ownpick(l):
    return l[randrange(0, len(l))]

for _ in range(r):
    print(choice(data), ownpick(data))  # Output: Random elements from data

5 19
24 67
24 67
19 67
5 5
24 19
24 5
5 24
5 24
5 24


6. **Weighted Random Selection**

**Using Weights for Random Selection**


In [8]:
weights = [6, 1, 5, 3]
from random import choices

print(choices(data, weights, k=2))  # Output: Two elements chosen based on weights

[5, 5]


**Custom Weighted Selection Function**

In [9]:
def pickweight(l, w):
    high = sum(w)
    v = myown(0, high)
    accumulated = 0
    for i, weight in enumerate(w):
        accumulated += weight
        if accumulated >= v:
            return l[i]

for _ in range(r):
    print(pickweight(data, weights))  # Output: Random weighted selection

67
67
67
67
24
24
5
5
67
5


7. **Sets and Permutations**

**Working with Sets**


In [10]:
# Example: Using sets to avoid duplicates
goal = 2
gotten = set()
while len(gotten) < goal:
    gotten.add(pickweight(data, weights))

print(gotten)  # Output: Unique elements from data

{67, 5}


**Shuffling a List**

In [11]:
from random import shuffle

shuffle(data)
print(data)  # Output: Shuffled list

[24, 5, 67, 19]


**Conclusion**

In this module, we explored the concepts of pseudorandomness, sets and permutations. We learned how to generate random numbers and values, select random elements from lists, and perform weighted random selection. We also delved into the use of sets for unique selections and shuffling lists. Understanding these concepts will enhance your ability to handle randomness calculations in your programs.


